# MIT Python Course 6.0001<br>

## Problem Set 3
### Problem Set 3, Part 5: Playing a hand (scroll down)<br>


This game is a lot like Scrabble or Words With Friends. Letters are dealt to players, who then construct one or more words using their letters. Each valid word earns the user points, based on the length of the word and the letters in that word.<br>

The rules of the game are as follows.<br>

**Dealing**<br>
> - A player is dealt a hand of `HAND_SIZE` letters of the alphabet, chosen at random.This may include multiple instances of a particular letter.<br>
> - The player arranges the hand into as many words as they want out of the letters, but using each letter at most once.<br>
> - Some letters may remain unused, though the size of the hand when a word is played does affect its score.<br>

**Scoring**<br>
> - The score for the hand is the sum of the score for each word formed.<br>
> - The score for a word is the product of two components:
>  - First component: the sum of the points for letters in the word.<br>
>  - Second component: either _[7 * word_length - 3 * (n - word_length)]_ or _1_, whichever value is greater, where:<br>
>    - `word_length` is the number of letters used in the word<br>
>    - `n` is the number of letters available in the current hand<br>
> - Letters are scored as in Scrabble; A is worth 1, B is worth 3, C is worth 3, D is worth 2, E is worth 1, and so on. We have defined the dictionary `SCRABBLE_LETTER_VALUES` that maps each lowercase letter to its Scrabble letter value.<br>
> - Examples:<br>
>   - For example, if _n=6_ and the hand includes 1 'w', 2 'e's, and 1 'd' (as well as two other letters), playing the word 'weed' would be worth 176 points: _(4+1+1+2) * (7*4 - 3*(6-4)) = 176_. The first term is the sum of the values of each letter used; the second term is the special computation that rewards a player for playing a longer word, and penalizes them for any left overletters.<br>
>   - As another example, if _n=7_, playing the word 'it' would be worth 2 points: *(1+1) * (1) = 2*. The second component is 1 because *7*2 - 3*(7 - 2) = -1*,which is less than 1.
***

**Getting Started**<br>
1. Files to be used are as follows:<br>
> - File `ps3.py` should contain all of the code and provides a set of  procedures.<br>
> - File `test_ps3.py` is for testing the code.<br>
> - File `words.txt` contains the legitimate words.<br>
2. Runing `ps3.py` loads a list of valid wordsfrom a file and then calls the `play_game` function. If everything is okay, after a small delay, you should see the following printed out:
`Loading word list from file...`<br>
`83667 words loaded.`<br>
`play_game not yet implemented.`<br> 
If you see an `IOError` instead (e.g., No such file or directory), make sure you have saved `words.txt` in the same directory as `ps3.py`!<br>
3. The file `ps3.py` has a number of already-implemented functions (Helper code).<br>
4. In this problem set a number of modular functions were written and then glued together to form the complete game. Instead of waiting until the entire game is ready, you should test each function you write, individually, before moving on. This approach is known as **unit testing**, and it will help you debug yourcode.<br>
5. There are several test functions to get you started. As you make progress on the problem set, run `test_ps3.py` to check your work so far. If your code passes the **unit tests** you will see a `SUCCESS` message; otherwise you will see a `FAILURE` message. These tests aren't exhaustive. You may want to test your code in other ways too (for example, with different test values) . These are the provided test functions:<br>
> `test_get_word_score` tests the `get_word_score`<br>
> `implementation.test_update_hand` tests the `update_hand`<br>
> `implementation.test_is_valid_word` tests the `is_valid_word`<br>
> `implementation.test_wildcard` testa the modifications made to support wildcards. (more about those later on)<br>
***

#### <font color = red>Problem Set 3, Part 5: Playing a hand</font><br>

Implement the `play_hand` function. This function allows the user to play out a single hand. You'll first need to implement the helper function `calculate_handlen` , which can be done in under five lines of code.<br>

To end the hand early, the player must type " `!!` " (two exclamation points).<br>

Note that after the line `# BEGIN PSEUDOCODE` there is a bunch of, well, pseudocode!<br>

**Testing:** Try out your implementation as if you were playing the game: run your program and call the `play_hand` function from your shell with a hand and the `word_list`.

**Note:** Your output should match the examples below. You should not print extraneous " None " messages.

##Example #1##<br>
> `Current Hand: a j e f * r x`<br>
`Enter word, or "!!" to indicate that you are finished:` <font color = blue>jar</font><br> 
`"jar" earned 90 points. Total: 90 points`<br>
<br>
`Current Hand: * f x e`<br>
`Enter word, or "!!" to indicate that you are finished:` <font color = blue>f*x</font><br> 
`"f * x" earned 216 points. Total: 306 points`<br>
<br>
`Current Hand: e`<br>
`Enter word, or "!!" to indicate that you are finished:` <font color = blue>!!</font><br> 
`Total score: 306 points`<br> 

**Example #2**<br>
> `Current Hand: a c f i * t x `<br>
`Enter word, or "!!" to indicate that you are finished:` <font color = blue>fix</font><br>
`"fix" earned 117 points. Total: 117 points`<br>
<br>
`Current Hand: a c t *`<br>
`Enter word, or "!!" to indicate that you are finished:` <font color = blue>ac</font><br>
`That is not a valid word. Please choose another word.`<br>
<br>
`Current Hand: t *`<br>
`Enter word, or "!!" to indicate that you are finished:` <font color = blue>*t</font><br>
`" * t" earned 14 points. Total: 131 points`<br>
<br>
`Ran out of letters. Total score: 131 points`<br>
***

In [5]:
import math
import random
import string

VOWELS = 'aeiou'
VOWELS_WITH_WILD = VOWELS + '*' # "*" is a wildcard
CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
HAND_SIZE = 7

SCRABBLE_LETTER_VALUES = {
    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, 'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1, 'o': 1, 'p': 3, 'q': 10, 'r': 1, 's': 1, 't': 1, 'u': 1, 'v': 4, 'w': 4, 'x': 8, 'y': 4, 'z': 10
}

WORDLIST_FILENAME = "words.txt"

def load_words():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # wordlist: list of strings
    wordlist = []
    for line in inFile:
        wordlist.append(line.strip().lower())
    print("  ", len(wordlist), "words loaded.")
    return wordlist


def get_frequency_dict(sequence):
    """
    Returns a dictionary where the keys are elements of the sequence
    and the values are integer counts, for the number of times that
    an element is repeated in the sequence.

    sequence: string or list
    return: dictionary
    """
    
    # freqs: dictionary (element_type -> int)
    freq = {}
    for x in sequence:
        freq[x] = freq.get(x,0) + 1
    return freq

def get_word_score(word, n):
    """
    Returns the score for a word. Assumes the word is a
    valid word.

    You may assume that the input word is always either a string of letters, 
    or the empty string "". You may not assume that the string will only contain 
    lowercase letters, so you will have to handle uppercase and mixed case strings 
    appropriately. 

	The score for a word is the product of two components:

	The first component is the sum of the points for letters in the word.
	The second component is the larger of:
            1, or
            7*wordlen - 3*(n-wordlen), where wordlen is the length of the word
            and n is the hand length when the word was played

	Letters are scored as in Scrabble; A is worth 1, B is
	worth 3, C is worth 3, D is worth 2, E is worth 1, and so on.

    word: string
    n: int >= 0
    returns: int >= 0
    """
    
    first_component = 0
    second_component = 0
    
    word = word.lower()
    
    for char in word:
        first_component += SCRABBLE_LETTER_VALUES.get(char, 0)
    
    wordlen = len(word)
    second_component = max(1, 7*wordlen - 3*(n-wordlen))
    
    return first_component*second_component

def display_hand(hand):
    """
    Displays the letters currently in the hand.

    For example:
       display_hand({'a':1, 'x':2, 'l':3, 'e':1})
    Should print out something like:
       a x x l l l e
    The order of the letters is unimportant.

    hand: dictionary (string -> int)
    """
    
    for letter in hand.keys():
        for j in range(hand[letter]):
             print(letter, end=' ')      # print all on the same line
    print()                              # print an empty line


def deal_hand(n):
    """
    Returns a random hand containing n lowercase letters.
    ceil(n/3) letters in the hand should be VOWELS (note,
    ceil(n/3) means the smallest integer not less than n/3).

    Hands are represented as dictionaries. The keys are
    letters and the values are the number of times the
    particular letter is repeated in that hand.

    n: int >= 0
    returns: dictionary (string -> int)
    """
    
    hand={}
    num_vowels = int(math.ceil(n / 3))
    
    for i in range(num_vowels): 
        x = random.choice(VOWELS_WITH_WILD)
        if i == num_vowels - 1 and hand.get('*',0) == 0:
            x= '*'
        hand[x] = hand.get(x, 0) + 1
    
    for i in range(num_vowels, n):    
        x = random.choice(CONSONANTS)
        hand[x] = hand.get(x, 0) + 1
    
    return hand

def update_hand(hand, word):
    """
    Does NOT assume that hand contains every letter in word at least as
    many times as the letter appears in word. Letters in word that don't
    appear in hand should be ignored. Letters that appear in word more times
    than in hand should never result in a negative count; instead, set the
    count in the returned hand to 0 (or remove the letter from the
    dictionary, depending on how your code is structured). 

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.

    word: string
    hand: dictionary (string -> int)    
    returns: dictionary (string -> int)
    """
    hand_copy = hand.copy()
    for word_letter in word:        
        for hand_letter in hand_copy.keys():
            if word_letter == hand_letter:
                hand_copy[hand_letter] -= 1
                if hand_copy[hand_letter] == 0:
                    del hand_copy[hand_letter]
                break
    return hand_copy
    


def is_valid_word(word, hand, word_list):
    """
    Returns True if word is in the word_list and is entirely
    composed of letters in the hand. Otherwise, returns False.
    Does not mutate hand or word_list.
   
    word: string, input by user
    hand: dictionary (string : int)
    word_list: list of lowercase strings
    returns: boolean
    """
        
    def hand_contains_word(word, hand):
        """
        Retruns true of all the letters of 'word' exit in 'hand'
        
        word: string, input by user
        hand: dictionary (string:int)
        returns: boolean
        """
        
        hand_copy = hand.copy()
        
        for word_letter in word:
            initial_sum_of_hand_values = sum(hand_copy.values())
            for hand_letter in hand_copy.keys():
                if word_letter == hand_letter:
                    hand_copy[hand_letter] -= 1
                    if hand_copy[hand_letter] == 0:
                        del hand_copy[hand_letter]
                    break
            updated_sum_of_hand_values = sum(hand_copy.values())
            if updated_sum_of_hand_values == initial_sum_of_hand_values:
                return False
             
        return True
    
    def wordlist_contains_word_with_wildcard (word, word_list):
        """
        Reaplce the wildcard (*) with one the vowels and ...
        returns True if the resulting word exist in word_list
        
        word: string, inpiut by user that contains a wildcard (*)
        word_list: list[string], all possible valid words
        """
        
        for vowel in VOWELS:
            word_copy = word.replace('*',vowel)
            if (word_copy in word_list):
                return True
        return False
        
        
    word = str.lower(word)
    if (word in word_list or wordlist_contains_word_with_wildcard (word, word_list)) and hand_contains_word(word, hand):
        return True
    return False


def calculate_handlen(hand):
    """ 
    Returns the length (number of letters) in the current hand.
    
    hand: dictionary (string-> int)
    returns: integer
    """
    
    handlen = 0
    for key in hand.keys():
        handlen += hand[key]
    return handlen
    


def play_hand(hand, word_list):

    """
    Allows the user to play the given hand, as follows:

    * The hand is displayed.
    
    * The user may input a word.

    * When any word is entered (valid or invalid), it uses up letters
      from the hand.

    * An invalid word is rejected, and a message is displayed asking
      the user to choose another word.

    * After every valid word: the score for that word is displayed,
      the remaining letters in the hand are displayed, and the user
      is asked to input another word.

    * The sum of the word scores is displayed when the hand finishes.

    * The hand finishes when there are no more unused letters.
      The user can also finish playing the hand by inputing two 
      exclamation points (the string '!!') instead of a word.

      hand: dictionary (string -> int)
      word_list: list of lowercase strings
      returns: the total score for the hand
      
    """
    
    total_score = 0
    # Keep track of the total score
    while calculate_handlen(hand) > 0:
    # As long as there are still letters left in the hand:
    
        # Display the hand
        print()
        print('Current Hand:', end = ' ')
        display_hand(hand)
        
        # Ask user for input
        word = input('Enter word, or "!!" to indicate that you are finished: ')
        word = str.lower(word)
        
        # If the input is two exclamation points:
        if word == '!!':
            
            break  # End the game (break out of the loop)
        
        # Otherwise (the input is not two exclamation points):
        elif is_valid_word(word, hand, word_list):  # If the word is valid:
            
            # Tell the user how many points the word earned,
            # and the updated total score
            score = get_word_score(word, calculate_handlen(hand))
            total_score += score

            print('"', word,'" earned', score, 'points. Total:', total_score, 'points')
            
        
        else:

            # Otherwise (the word is not valid):
            # Reject invalid word (print a message)    
            print('That is not a valid word. Please choose another word.')
            
                
        # update the user's hand by removing the letters of their inputted word
        hand  = update_hand(hand, word)
            

    # Game is over (user entered '!!' or ran out of letters),
    # so tell user the total score
    if word == '!!':
        print('Total score:', total_score, 'points')
    else:
        print()
        print('Ran out of letters. Total score:', total_score, 'points')

    # Return the total score as result of function
    return total_score


if __name__ == '__main__':
    word_list = load_words()
    # hand = {'a':1, 'm':3, 'n':1, '*':1}
    # hand = {'a':1, 'j':1, 'e':1, 'f':1, '*':1, 'r':1, 'x':1}
    hand = {'a':1, 'c':1, 'f':1, 'i':1, '*':1, 't':1, 'x':1}
    play_hand(hand, word_list)

    

Loading word list from file...
   83667 words loaded.

Current Hand: a c f i * t x 
Enter word, or "!!" to indicate that you are finished: fix
" fix " earned 117 points. Total: 117 points

Current Hand: a c * t 
Enter word, or "!!" to indicate that you are finished: ac
That is not a valid word. Please choose another word.

Current Hand: * t 
Enter word, or "!!" to indicate that you are finished: *t
" *t " earned 14 points. Total: 131 points

Ran out of letters. Total score: 131 points
